In [19]:
import numpy as np
import cv2
import socketio
import base64
import winsound as sd

In [20]:
def GetPerson(frame, size, score_threshold, nms_threshold):
    # 이미지의 높이, 너비, 채널 받아오기
    height, width, channels = frame.shape

    # 네트워크에 넣기 위한 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (size, size), (0, 0, 0), True, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(blob)

    # 결과 받아오기
    outs = net.forward(output_layers)

    # 각각의 데이터를 저장할 빈 리스트
    class_ids = []
    confidences = []
    boxes = []
    
    iCntPerson = 0

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            
            #사람일 경우만 객체에 담기
            if class_id == 0:
                confidence = scores[class_id]

                if confidence > 0.7:
                    # 탐지된 객체의 너비, 높이 및 중앙 좌표값 찾기
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # 객체의 사각형 테두리 중 좌상단 좌표값 찾기
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
                    
                    iCntPerson = iCntPerson + 1

    # Non Maximum Suppression (겹쳐있는 박스 중 confidence 가 가장 높은 박스를 선택)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=score_threshold, nms_threshold=nms_threshold)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            class_name = classes[class_ids[i]]
            
            label = f"{class_name} {confidences[i]:.2f}"            
            color = [2, 173, 255]
            # 사각형 테두리 그리기 및 텍스트 쓰기
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.rectangle(frame, (x - 1, y), (x + len(class_name) * 13 + 65, y - 25), color, -1)
            cv2.putText(frame, label, (x, y - 8), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 0), 2)
    return frame, iCntPerson

In [21]:
"""
@는 데코레이터임
https://bluese05.tistory.com/30 설명
"""
imdata = 0
sio = socketio.Client()

@sio.event
def streaming(data):
    #받아온 데이터를 Decode 처리
    data = cv2.imdecode(np.frombuffer(base64.b64decode(data), dtype='uint8'), cv2.IMREAD_COLOR)
    global imdata
    imdata = data

In [22]:
#Yolo 학습결과 레이어 불러오기
net = cv2.dnn.readNet(f"./Yolo/yolov3_320.weights", "./Yolo/yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# 클래스 리스트
classes = []
with open("./Yolo/yolo.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
# 입력 사이즈 리스트 (Yolo 에서 사용되는 네크워크 입력 이미지 사이즈)
size_list = [320, 416, 608]

#소켓통신을 위한 준비
isSocket = True

#소켓 호출 에러시 처리하기 위한 부분
try:
    sio.connect('http://localhost:746')
except:
    isSocket = False

if isSocket == True:
    while True:
        sio.on('streaming', streaming)
        
        if imdata is not None:        
            frame, iCntPerson = GetPerson(frame=imdata, size=size_list[0], score_threshold=0.4, nms_threshold=0.4)        
            if iCntPerson > 0:
                #소켓으로 주의 값 전송
                sio.emit('wop', iCntPerson)
            cv2.imshow('Person Detect', frame)
        else:
            cv2.imshow('Person Detect', imdata)
        
        if cv2.waitKey(1) == 13:
            break
            
    sio.disconnect()
    cv2.destroyAllWindows()
else:
    print("소켓접속 불량")

In [ ]:
#cv2.HOGDescriptor_getDefaultPeopleDetector <-- 이걸로 검출해보기